In [14]:
import random
import numpy as np

In [15]:
def sigmoid(z):
    """
    The sigmoid function.

    Parameters:
    z (numpy.ndarray): The input to the sigmoid function.

    Returns:
    numpy.ndarray: The sigmoid of the input.
    """
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    """
    Derivative of the sigmoid function.

    Parameters:
    z (numpy.ndarray): The input to the sigmoid function.

    Returns:
    numpy.ndarray: The derivative of the sigmoid function at z.
    """
    return sigmoid(z) * (1 - sigmoid(z))

## 3-layer NN
![This is an example image](http://neuralnetworksanddeeplearning.com/images/tikz10.png)

## Weights initilization

In [6]:
# Size contains the number of neurons in the respective layers
sizes = [3, 4, 1]
# Understand dimensions of weights
for x, y in zip(sizes[:-1], sizes[1:]):
    print(x, y)
# Initialize weights using Gaussian distribution with mean 0 and standard deviation 1
weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
print(weights)

3 4
4 1
[array([[-0.46651846, -1.12942651,  0.19766863],
       [ 2.40491053,  0.33854765,  0.67934898],
       [-1.54463351, -1.82830397, -0.04256603],
       [ 0.13525649, -0.10129711,  0.34887896]]), array([[ 2.23689866, -0.44323522, -0.05001511,  1.58402228]])]


For the above network, the number of weights for each neuron in a layer is determined by the number of neurons in the previous layer. For instance, the second layer (i.e., hidden layer) in the above 3-layer NN has four neurons. For each neuron in this layer there are three inputs from the three neurons in the previous layer, which requires three weights. Therefore, the dimension of the array to store all the weights for this layer has shape of (4, 3), with the first dimension corresponds to the number of neurons in the current layer and second dimension corresponds to the total number of neurons in the previous layer.

## Bias initialization

In [8]:
# Understand dimensions of bias
for y in sizes[1:]:
    print(y)
# Initialize bias using Gaussian distribution with mean 0 and standard deviation 1    
bias = [np.random.randn(y, 1) for y in sizes[1:]]
print(bias)

4
1
[array([[ 1.5446769 ],
       [ 0.04072502],
       [ 1.05776775],
       [-0.81686915]]), array([[1.07602797]])]


For bias, firstly no bias term is needed for the input layer since the neurons in this layer do not have any outputs coming in. The number of bias terms for all the subsequent layes, including output layer, is equal to the number of neurons in each layer, which is captured in the sizes of the network.

In [9]:
class Network:
    def __init__(self, sizes):
        """
        Initialize the neural network with the given sizes.
        
        Parameters:
        sizes (list): A list containing the number of neurons in each layer.
                      For example, [784, 30, 10] would create a network with
                      784 input neurons, one hidden layer with 30 neurons,
                      and an output layer with 10 neurons.
        """
        self.num_layers = len(sizes)
        self.sizes = sizes
        # Biases and weights are initialized with Gaussian distribution
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """
        Return the output of the network if 'a' is input.
        
        Parameters:
        a (numpy array): The input to the network.
        
        Returns:
        numpy array: The output of the network.
        """
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """
        Train the neural network using mini-batch stochastic gradient descent.
        
        Parameters:
        training_data (list): A list of tuples '(x, y)' representing the training inputs
                              and the desired outputs.
        epochs (int): The number of epochs to train for.
        mini_batch_size (int): The size of the mini-batches to use when sampling.
        eta (float): The learning rate.
        test_data (list, optional): If provided, the network will be evaluated
                                    against the test data after each epoch, and
                                    partial progress will be printed out.
        """
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print(f"Epoch {j}: {self.evaluate(test_data)} / {n_test}")
            else:
                print(f"Epoch {j} complete")

    def update_mini_batch(self, mini_batch, eta):
        """
        Update the network's weights and biases by applying gradient descent
        using backpropagation to a single mini-batch.
        
        Parameters:
        mini_batch (list): A list of tuples '(x, y)'.
        eta (float): The learning rate.
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w - (eta / len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta / len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """
        Return a tuple representing the gradient for the cost function.
        
        Parameters:
        x (numpy array): The input to the network.
        y (numpy array): The desired output.
        
        Returns:
        tuple: (nabla_b, nabla_w), representing the gradients for the biases and weights.
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # Feedforward
        activation = x
        activations = [x]  # List to store all the activations, layer by layer
        zs = []  # List to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # Backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Update the gradients for the previous layers
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """
        Return the number of test inputs for which the neural network outputs
        the correct result.
        
        Parameters:
        test_data (list): A list of tuples '(x, y)' where 'x' is the input and 'y' is the desired output.
        
        Returns:
        int: The number of test inputs for which the network is correct.
        """
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """
        Return the vector of partial derivatives ∂C/∂a for the output activations.
        
        Parameters:
        output_activations (numpy array): The output of the network.
        y (numpy array): The desired output.
        
        Returns:
        numpy array: The derivative of the cost function.
        """
        return (output_activations - y)

## Create a Network object

In [12]:
# A neural network with three layers: one input layer with three neurons, one hidden layer with four neurons and 
# one output layer with one neuron
net = Network([3, 4, 1])
# Weights connecting the second layer and third layers of neurons
net.weights[1].shape

(1, 4)

## Understand `feedforward` function of `Network` object
Let's walk through an example of the `feedforward` function using a neural network with sizes `[3, 4, 1]`. This means the network has:
- 3 neurons in the input layer
- 4 neurons in the hidden layer
- 1 neuron in the output layer

### Example

For the sake of this example, let's assume the following initial random weights and biases for the network:

#### Weights
- Between input layer and hidden layer (3 input neurons to 4 hidden neurons):
  ```python
  w1 = [[0.2, -0.5, 1.0],
        [1.5, -1.0, 0.5],
        [-1.5, 2.0, -1.0],
        [0.5, -0.5, 1.5]]
  ```
- Between hidden layer and output layer (4 hidden neurons to 1 output neuron):
  ```python
  w2 = [[0.3, -0.8, 0.5, 1.0]]
  ```

#### Biases
- For hidden layer (4 neurons):
  ```python
  b1 = [[0.1],
        [0.2],
        [0.3],
        [0.4]]
  ```
- For output layer (1 neuron):
  ```python
  b2 = [[0.5]]
  ```

### Input
Input layer of this NN has three neurons. Let's take a specific input training example for those three input neurons:
```python
input_vector = [[0.5],
                [0.1],
                [0.4]]
```

In [46]:
%%html
<svg width="800" height="400" viewBox="0 0 800 400" xmlns="http://www.w3.org/2000/svg">
  <svg viewBox="0 0 800 400" xmlns="http://www.w3.org/2000/svg">
  <!-- Input Layer -->
  <g id="input-layer">
    <circle cx="100" cy="80" r="20" fill="lightblue" stroke="black"/>
    <text x="100" y="85" text-anchor="middle" font-size="12">0.5</text>
    <circle cx="100" cy="160" r="20" fill="lightblue" stroke="black"/>
    <text x="100" y="165" text-anchor="middle" font-size="12">0.1</text>
    <circle cx="100" cy="240" r="20" fill="lightblue" stroke="black"/>
    <text x="100" y="245" text-anchor="middle" font-size="12">0.4</text>
  </g>
  
  <!-- Hidden Layer -->
  <g id="hidden-layer">
    <circle cx="300" cy="60" r="20" fill="lightgreen" stroke="black"/>
    <text x="300" y="65" text-anchor="middle" font-size="12">0.63</text>
    <circle cx="300" cy="140" r="20" fill="lightgreen" stroke="black"/>
    <text x="300" y="145" text-anchor="middle" font-size="12">0.74</text>
    <circle cx="300" cy="220" r="20" fill="lightgreen" stroke="black"/>
    <text x="300" y="225" text-anchor="middle" font-size="12">0.34</text>
    <circle cx="300" cy="300" r="20" fill="lightgreen" stroke="black"/>
    <text x="300" y="305" text-anchor="middle" font-size="12">0.77</text>
  </g>
  
  <!-- Output Layer -->
  <g id="output-layer">
    <circle cx="500" cy="160" r="20" fill="lightyellow" stroke="black"/>
    <text x="500" y="165" text-anchor="middle" font-size="12">0.74</text>
  </g>
  
  <!-- Connections -->
  <g id="connections" stroke="gray" stroke-width="1">
    <!-- Input to Hidden -->
    <line x1="120" y1="80" x2="280" y2="60"/>
    <line x1="120" y1="80" x2="280" y2="140"/>
    <line x1="120" y1="80" x2="280" y2="220"/>
    <line x1="120" y1="80" x2="280" y2="300"/>
    
    <line x1="120" y1="160" x2="280" y2="60"/>
    <line x1="120" y1="160" x2="280" y2="140"/>
    <line x1="120" y1="160" x2="280" y2="220"/>
    <line x1="120" y1="160" x2="280" y2="300"/>
    
    <line x1="120" y1="240" x2="280" y2="60"/>
    <line x1="120" y1="240" x2="280" y2="140"/>
    <line x1="120" y1="240" x2="280" y2="220"/>
    <line x1="120" y1="240" x2="280" y2="300"/>
    
    <!-- Hidden to Output -->
    <line x1="320" y1="60" x2="480" y2="160"/>
    <line x1="320" y1="140" x2="480" y2="160"/>
    <line x1="320" y1="220" x2="480" y2="160"/>
    <line x1="320" y1="300" x2="480" y2="160"/>
  </g>
  
  <!-- Weights -->
  <g id="weights" font-size="10" fill="red">
    <!-- Input to Hidden -->
    <text x="150" y="70">0.2</text>
    <text x="150" y="90">1.5</text>
    <text x="150" y="110">-1.5</text>
    <text x="150" y="130">0.5</text>
    
    <text x="180" y="100">-0.5</text>
    <text x="180" y="150">-1.0</text>
    <text x="180" y="180">2.0</text>
    <text x="180" y="210">-0.5</text>
    
    <text x="230" y="100">1.0</text>
    <text x="230" y="160">0.5</text>
    <text x="230" y="220">-1.0</text>
    <text x="230" y="280">1.5</text>
    
    <!-- Hidden to Output -->
    <text x="400" y="100">0.3</text>
    <text x="400" y="140">-0.8</text>
    <text x="400" y="180">0.5</text>
    <text x="400" y="220">1.0</text>
  </g>
  
  <!-- Labels -->
  <text x="100" y="20" text-anchor="middle" font-size="14" font-weight="bold">Input Layer</text>
  <text x="300" y="20" text-anchor="middle" font-size="14" font-weight="bold">Hidden Layer</text>
  <text x="500" y="20" text-anchor="middle" font-size="14" font-weight="bold">Output Layer</text>
  <text x="600" y="160" font-size="14">Output: 0.74</text>
</svg>
</svg>

### Feedforward Calculation

Let's break down the `feedforward` function step by step:

1. **Initialization**: Set the input vector `a` as the initial activation.

    ```python
    a = input_vector
    ```

2. **First Layer Calculation (Input to Hidden)**:
   - Compute the weighted input $z_1$ for the hidden layer:
     \begin{equation}
     z_1 = w_1 \cdot a + b_1
     \end{equation}
     Substituting the values:
     $$
     z_1 = \begin{bmatrix}
     0.2 & -0.5 & 1.0 \\
     1.5 & -1.0 & 0.5 \\
     -1.5 & 2.0 & -1.0 \\
     0.5 & -0.5 & 1.5 
     \end{bmatrix} \cdot \begin{bmatrix}
     0.5 \\
     0.1 \\
     0.4 
     \end{bmatrix} + \begin{bmatrix}
     0.1 \\
     0.2 \\
     0.3 \\
     0.4 
     \end{bmatrix}
     $$
     
     Calculate the dot product:
     
     $$
     w_1 \cdot a = \begin{bmatrix}
     0.2 \cdot 0.5 + -0.5 \cdot 0.1 + 1.0 \cdot 0.4 \\
     1.5 \cdot 0.5 + -1.0 \cdot 0.1 + 0.5 \cdot 0.4 \\
     -1.5 \cdot 0.5 + 2.0 \cdot 0.1 + -1.0 \cdot 0.4 \\
     0.5 \cdot 0.5 + -0.5 \cdot 0.1 + 1.5 \cdot 0.4
     \end{bmatrix} = \begin{bmatrix}
     0.1 + -0.05 + 0.4 \\
     0.75 + -0.1 + 0.2 \\
     -0.75 + 0.2 + -0.4 \\
     0.25 + -0.05 + 0.6
     \end{bmatrix} = \begin{bmatrix}
     0.45 \\
     0.85 \\
     -0.95 \\
     0.8 
     \end{bmatrix}
     $$
     Adding the bias:
     $$
     z_1 = \begin{bmatrix}
     0.45 \\
     0.85 \\
     -0.95 \\
     0.8 
     \end{bmatrix} + \begin{bmatrix}
     0.1 \\
     0.2 \\
     0.3 \\
     0.4 
     \end{bmatrix} = \begin{bmatrix}
     0.55 \\
     1.05 \\
     -0.65 \\
     1.2 
     \end{bmatrix}
     $$

   - Apply the sigmoid activation function to get the activation $a_1$ of the hidden layer:
     $$
     a_1 = \sigma(z_1) = \sigma\left(\begin{bmatrix}
     0.55 \\
     1.05 \\
     -0.65 \\
     1.2 
     \end{bmatrix}\right) = \begin{bmatrix}
     \sigma(0.55) \\
     \sigma(1.05) \\
     \sigma(-0.65) \\
     \sigma(1.2)
     \end{bmatrix} = \begin{bmatrix}
     0.63413559 \\
     0.7407749 \\
     0.34298954 \\
     0.76852478
     \end{bmatrix}
     $$

3. **Second Layer Calculation (Hidden to Output)**:
   - Compute the weighted input $z_2$ for the output layer:
     $$
     z_2 = w_2 \cdot a_1 + b_2
     $$
     
     Substituting the values:
     
     $$
     z_2 = \begin{bmatrix}
     0.3 & -0.8 & 0.5 & 1.0
     \end{bmatrix} \cdot \begin{bmatrix}
     0.63413559 \\
     0.7407749 \\
     0.34298954 \\
     0.76852478
     \end{bmatrix} + \begin{bmatrix}
     0.5
     \end{bmatrix}
     $$
     Calculate the dot product:
     $$
     w_2 \cdot a_1 = 0.3 \cdot 0.63413559 + -0.8 \cdot 0.7407749 + 0.5 \cdot 0.34298954 + 1.0 \cdot 0.76852478 = 0.19024068 + -0.59261992 + 0.17149477 + 0.76852478 = 0.53764031
     $$
     Adding the bias:
     $$
     z_2 = 0.53764031 + 0.5 = 1.03764031
     $$

   - Apply the sigmoid activation function to get the activation `a2` of the output layer:
     $$
     a_2 = \sigma(z_2) = \sigma(1.03764031) = 0.73841914
     $$

### Final Output
So, given the input vector `[[0.5], [0.1], [0.4]]`, the network produces an output of `0.73841914`.

### Summary
- **Input Layer**: Receives the input `[[0.5], [0.1], [0.4]]`.
- **Hidden Layer**: 
  - Computes $z_1$ using weights and biases.
  - Applies the sigmoid function to $z_1$ to get $a_1$.
- **Output Layer**:
  - Computes $z_2$ using weights and biases.
  - Applies the sigmoid function to $z_2$ to get $a_2$.

This step-by-step example demonstrates how the `feedforward` function processes an input through the layers of the network to produce the final output.

In [47]:
# Create a network with the same structure as our previous example
net = Network([3, 4, 1])

# Set the weights and biases to match our previous example
net.weights = [
    np.array([[0.2, -0.5, 1.0],
              [1.5, -1.0, 0.5],
              [-1.5, 2.0, -1.0],
              [0.5, -0.5, 1.5]]),
    np.array([[0.3, -0.8, 0.5, 1.0]])
]

net.biases = [
    np.array([[0.1], [0.2], [0.3], [0.4]]),
    np.array([[0.5]])
]

# Input vector
input_vector = np.array([[0.5], [0.1], [0.4]])

# Use the feedforward method
output = net.feedforward(input_vector)

print("Network Feedforward Result:")
print(f"Input: {input_vector.T}")
print(f"Output: {output[0][0]:.8f}")

# Compare with our previous manual calculation
print("\nComparison:")
print(f"Manual calculation result: 0.73841914")
print(f"Network feedforward result: {output[0][0]:.8f}")
print(f"Difference: {abs(0.73841914 - output[0][0]):.8f}")

Network Feedforward Result:
Input: [[0.5 0.1 0.4]]
Output: 0.73839445

Comparison:
Manual calculation result: 0.73841914
Network feedforward result: 0.73839445
Difference: 0.00002469


In [51]:
# Create a network with the same structure as our previous example
# The weights and biases are initialized by the Network object.
net = Network([3, 4, 1])

# Input vector
input_vector = np.array([[0.5], [0.1], [0.4]])

# Use the feedforward method
output = net.feedforward(input_vector)

print("Network Feedforward Result:")
print(f"Input: {input_vector.T}")
print(f"Output: {output[0][0]:.8f}")

Network Feedforward Result:
Input: [[0.5 0.1 0.4]]
Output: 0.50672075
